In [1]:
import requests
import time
import os, json
import csv
import concurrent.futures
from functools import partial
import pandas as pd

In [2]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_5Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['Response', 'Response_status', 'Time (seconds)'])

In [3]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [4]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            yield line.strip()

In [5]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Response':result.text, 'Response_status':result.status, 'Time (seconds)':f'{execution_time:.6f}'}
        df.append(data, ignore_index=True)
        print(f"Execution time of {result}: {execution_time:.6f} seconds - Response===> {_.text} \n\n")
        return result, _
    return calculate_execution_time

In [6]:
def save_report(file_path, df):
    file_exists = os.path.exists(file_path)
    df.to_csv(file_path, index=False)
#     with open(file_path, 'a', newline='') as csv_file:
#         csv_writer = csv.writer(csv_file)
#         if not file_exists:
#             csv_writer.writerow(data[0])
#         for row in data[1:]:
#             csv_writer.writerow(row)

In [11]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        return response
    except Exception as e:
        print(e)
        

#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines  {**payload, 'prompt': {prompt}}
    prompts = read_payloads(request_payload_file)
    for i in range(1):
        with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
            futures = [executor.submit(inference, payload) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

KeyboardInterrupt: 

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))